In [1]:
import tensorflow as tf
from tensorflow import keras

# Common imports
import numpy as np
import os
from glob import glob
from matplotlib import image
from numpy import asarray

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

In [2]:
train_dir = './datasets/Dog-Cat/train/'
test_dir  = './datasets/Dog-Cat/test1/'

In [10]:
# picture file name list
data_img_list = [os.path.basename(f) for f in glob(f'{train_dir}/*.jpg')]
test_img_list = [os.path.basename(f) for f in glob(f'{test_dir}/*.jpg')]

# make dog and cat file name list
img_cat_list=[]
img_dog_list=[]

for f in data_img_list:
    label = f.split('.')[0]
    if label == 'cat':
        img_cat_list.append(f)
    elif label == 'dog':
        img_dog_list.append(f)
    else:
        print('abnormal file name is found', f)

#img_dog_list=[img+'.jpg' for img in img_dog_list]
#img_cat_list=[img+'.jpg' for img in img_cat_list]

len(img_cat_list),len(img_dog_list)#classes are even

(12500, 12500)

In [11]:
# Split data for train and for valid, evenly across cats and dogs
from sklearn.model_selection import train_test_split
train_list_cat, val_list_cat, train_list_dog, val_list_dog = train_test_split(img_cat_list, img_dog_list, 
                                                                  test_size=0.3,
                                                                 random_state=46)
#create train and validation lists
train_list=train_list_cat+train_list_dog
val_list=val_list_cat+val_list_dog
lists=[train_list, val_list]


In [61]:
from keras.preprocessing.image import img_to_array,load_img,ImageDataGenerator

train_images=np.array([img_to_array(load_img(os.path.join(train_dir,img), target_size=(100,100))) for img in train_list])/255.
val_images=np.array([img_to_array(load_img(os.path.join(train_dir,img), target_size=(100,100))) for img in val_list])/255.
train_images.shape

(17500, 100, 100, 3)

In [57]:
#extract labels
train_y=[]
val_y=[]
lists_y=[train_y,val_y]
for l in [0,1]:
    for f in lists[l]:
        label = f.split('.')[0]
        if label == 'cat':
            lists_y[l].append(1)
        elif label == 'dog':
            lists_y[l].append(0)
        else:
            print('abnormal file name is found', f)
train_y=np.array(train_y)
val_y=np.array(val_y)

In [63]:
np.array(train_y).shape,np.array(val_y).shape,train_images.shape,val_images.shape

((17500,), (7500,), (17500, 100, 100, 3), (7500, 100, 100, 3))

In [35]:
model=keras.models.Sequential()

model.add(tf.keras.layers.Conv2D(64,(3,3),input_shape=(100,100,3)))
model.add(tf.keras.layers.Activation('relu'))

model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2)))

model.add(tf.keras.layers.Conv2D(32,(3,3)))
model.add(tf.keras.layers.Activation('relu'))

model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2)))

model.add(tf.keras.layers.Conv2D(16,(3,3)))
model.add(tf.keras.layers.Activation('relu'))

# model.add(MaxPool2D(pool_size=(2,2)))

model.add(tf.keras.layers.Conv2D(8,(3,3)))
model.add(tf.keras.layers.Activation('relu'))

model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dropout(0.25))

model.add(tf.keras.layers.Dense(100))
model.add(tf.keras.layers.Activation('relu'))

model.add(tf.keras.layers.Dense(100))
model.add(tf.keras.layers.Activation('relu'))

model.add(tf.keras.layers.Dense(100))
model.add(tf.keras.layers.Activation('relu'))

model.add(tf.keras.layers.Dropout(0.25))

model.add(tf.keras.layers.Dense(1))
model.add(tf.keras.layers.Activation('sigmoid'))

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 98, 98, 64)        1792      
_________________________________________________________________
activation_16 (Activation)   (None, 98, 98, 64)        0         
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 49, 49, 64)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 47, 47, 32)        18464     
_________________________________________________________________
activation_17 (Activation)   (None, 47, 47, 32)        0         
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 23, 23, 32)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 21, 21, 16)       

In [67]:
optimizer = tf.keras.optimizers.Adam(1e-5)

model.compile(optimizer='rmsprop',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

model.fit(train_images,train_y,
         validation_data=(val_images,val_y),
         batch_size=32,
         epochs=30,
         verbose=2)

In [55]:
model2 = keras.models.Sequential([
    keras.layers.Conv2D(32, kernel_size=3, padding="same", activation="relu"),
    keras.layers.Conv2D(64, kernel_size=3, padding="same", activation="relu"),
    keras.layers.MaxPool2D(),
    keras.layers.Flatten(),
    keras.layers.Dropout(0.25),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1, activation="sigmoid")
])
model2.compile(loss="categorical_crossentropy", optimizer='rmsprop',
              metrics=["accuracy"])
model2.fit(train_images, train_y , epochs=30, validation_data=(val_images,val_y))

In [ ]:
model3 = keras.models.Sequential()
model3.add( tf.keras.applications.EfficientNetB3(include_top=False, weights="imagenet", input_shape=(300,300,3)))
model3.add(tf.keras.layers.GlobalAveragePooling2D())
model3.add(tf.keras.layers.Dense(1, activation="sigmoid"))

opt = tf.keras.optimizers.Adam(learning_rate= 0.0001,beta_1=0.9, beta_2=0.999,epsilon=1e-07)
model3.compile(loss="categorical_crossentropy", optimizer=opt,
              metrics=["accuracy"])

model3.fit(train_images1, train_y1, epochs=30, validation_data=valid_data)

In [ ]:
resNet = tf.keras.applications.ResNet50(weights = 'imagenet',
                        include_top = False,
                        input_shape = (300,300, 3))

resNet.trainable = False # Freeze layers
resNet_model = Sequential([
        resNet,
        Flatten(),
        Dense(1024, activation = 'relu'),
        Dropout(0.4),
        Dense(1, activation = 'sigmoid')])
     

optimizer = optimizers.Adam(1e-5)

resNet_model.compile(optimizer = optimizer,
             loss = 'categorical_crossentropy',
             metrics = ['accuracy'])

resnet_history = resNet_model.fit((train_images, train_y), epochs = 30,
                          validation_data =(val_images, val_y))